In [1]:
import pandas as pd
from pathlib import Path
import pandas_datareader.data as reader
import datetime as dt
from MCForecastTools import MCSimulation

# For visualizations:
import seaborn as sns

# PART I: DATA CAPTURE AND ORGANIZATION

In [2]:
end = dt.date(2022, 5, 10)
start = dt.date(2022, 1, 1)

ticker1 = ["BTC-USD"]
ticker2 = ["IWP"]  #iShares Russell Mid-Cap GROWTH ETF, a midcap growth fund. Chose midcap to factor out size.
ticker3 = ["IWS"]  #iShares Russell Mid-Cap VALUE ETF, a midcap value fund. Chose midcap to factor out size.
ticker4 = ["MGC"]   #Vanguard Mega Cap ETF, a fund with LARGE CAPS only to feature size (large cap end of the spectrum).
ticker5 = ["IWC"]  #iShares Micro-Cap ETF, a fund with SMALL CAPS only to feature size (small cap end of the spectrum).

In [3]:
# Get BTC closing prices

btc_prices = reader.get_data_yahoo(ticker1, start, end)
display(btc_prices.head())
btc_prices.tail()

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,,
2022-01-01,47686.812500,47686.812500,47827.312500,46288.484375,46311.746094,24582667004
2022-01-02,47345.218750,47345.218750,47881.406250,46856.937500,47680.925781,27951569547
2022-01-03,46458.117188,46458.117188,47510.726562,45835.964844,47343.542969,33071628362
2022-01-04,45897.574219,45897.574219,47406.546875,45752.464844,46458.851562,42494677905
2022-01-05,43569.003906,43569.003906,46929.046875,42798.222656,45899.359375,36851084859


Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,,
2022-05-07,35501.953125,35501.953125,36129.925781,34940.824219,36042.503906,24375896406
2022-05-08,34059.265625,34059.265625,35502.941406,33878.964844,35502.941406,36763041910
2022-05-09,30296.953125,30296.953125,34222.074219,30296.953125,34060.015625,63355494961
2022-05-10,31022.906250,31022.906250,32596.308594,29944.802734,30273.654297,59811038817
2022-05-11,30659.839844,30659.839844,31988.117188,29367.916016,30985.792969,64206049280


In [4]:
btc_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
btc_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2022-01-01,47686.812500,47686.812500,47827.312500,46288.484375,46311.746094,24582667004
2022-01-02,47345.218750,47345.218750,47881.406250,46856.937500,47680.925781,27951569547
2022-01-03,46458.117188,46458.117188,47510.726562,45835.964844,47343.542969,33071628362
2022-01-04,45897.574219,45897.574219,47406.546875,45752.464844,46458.851562,42494677905
2022-01-05,43569.003906,43569.003906,46929.046875,42798.222656,45899.359375,36851084859


In [5]:
btc_prices = btc_prices[["open", "high", "low", "close", "volume", "adj close"]]
btc_prices.drop(columns="adj close", axis=1, inplace=True)
btc_prices.head()

,open,high,low,close,volume
Date,,,,,
2022-01-01,46311.746094,47827.312500,46288.484375,47686.812500,24582667004
2022-01-02,47680.925781,47881.406250,46856.937500,47345.218750,27951569547
2022-01-03,47343.542969,47510.726562,45835.964844,46458.117188,33071628362
2022-01-04,46458.851562,47406.546875,45752.464844,45897.574219,42494677905
2022-01-05,45899.359375,46929.046875,42798.222656,43569.003906,36851084859


In [6]:
btc_only_prices = pd.concat([btc_prices], axis=1, keys=['BTC USD'])
btc_only_prices

BTC USD                                            \
                    open          high           low         close   
Date                                                                 
2022-01-01  46311.746094  47827.312500  46288.484375  47686.812500   
2022-01-02  47680.925781  47881.406250  46856.937500  47345.218750   
2022-01-03  47343.542969  47510.726562  45835.964844  46458.117188   
2022-01-04  46458.851562  47406.546875  45752.464844  45897.574219   
2022-01-05  45899.359375  46929.046875  42798.222656  43569.003906   
...                  ...           ...           ...           ...   
2022-05-07  36042.503906  36129.925781  34940.824219  35501.953125   
2022-05-08  35502.941406  35502.941406  33878.964844  34059.265625   
2022-05-09  34060.015625  34222.074219  30296.953125  30296.953125   
2022-05-10  30273.654297  32596.308594  29944.802734  31022.906250   
2022-05-11  30985.792969  31988.117188  29367.916016  30659.839844   

                         
                 volume  
Date                     
2022-01-01  24582667004  
2022-01-02  27951569547  
2022-01-03  33071628362  
2022-01-04  42494677905  
2022-01-05  36851084859  
...                 ...  
2022-05-07  24375896406  
2022-05-08  36763041910  
2022-05-09  63355494961  
2022-05-10  59811038817  
2022-05-11  64206049280  

[131 rows x 5 columns]

In [7]:
iwp_prices = reader.get_data_yahoo(ticker2, start, end)
display(iwp_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,IWP,IWP,IWP,IWP,IWP,IWP
Date,,,,,,
2022-01-03,113.898430,114.029999,115.669998,113.230003,115.550003,1401700.0
2022-01-04,112.270309,112.400002,114.190002,111.059998,114.160004,1440300.0
2022-01-05,107.925331,108.050003,112.180000,108.019997,112.019997,2204800.0
2022-01-06,108.145073,108.269997,109.400002,106.559998,107.779999,2073000.0
2022-01-07,106.377121,106.500000,109.120003,106.459999,108.169998,2013200.0


In [8]:
iwp_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
iwp_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2022-01-03,113.898430,114.029999,115.669998,113.230003,115.550003,1401700.0
2022-01-04,112.270309,112.400002,114.190002,111.059998,114.160004,1440300.0
2022-01-05,107.925331,108.050003,112.180000,108.019997,112.019997,2204800.0
2022-01-06,108.145073,108.269997,109.400002,106.559998,107.779999,2073000.0
2022-01-07,106.377121,106.500000,109.120003,106.459999,108.169998,2013200.0


In [9]:
iwp_prices = iwp_prices[["open", "high", "low", "close", "volume", "adj close"]]
iwp_prices.drop(columns="adj close", axis=1, inplace=True)
iwp_prices.head()

,open,high,low,close,volume
Date,,,,,
2022-01-03,115.550003,115.669998,113.230003,114.029999,1401700.0
2022-01-04,114.160004,114.190002,111.059998,112.400002,1440300.0
2022-01-05,112.019997,112.180000,108.019997,108.050003,2204800.0
2022-01-06,107.779999,109.400002,106.559998,108.269997,2073000.0
2022-01-07,108.169998,109.120003,106.459999,106.500000,2013200.0


In [10]:
iwp_only_prices = pd.concat([iwp_prices], axis=1, keys=['IWP'])
iwp_only_prices

IWP                                               
                  open        high         low       close     volume
Date                                                                 
2022-01-03  115.550003  115.669998  113.230003  114.029999  1401700.0
2022-01-04  114.160004  114.190002  111.059998  112.400002  1440300.0
2022-01-05  112.019997  112.180000  108.019997  108.050003  2204800.0
2022-01-06  107.779999  109.400002  106.559998  108.269997  2073000.0
2022-01-07  108.169998  109.120003  106.459999  106.500000  2013200.0
...                ...         ...         ...         ...        ...
2022-05-04   90.230003   92.779999   88.239998   92.660004  1767500.0
2022-05-05   91.639999   91.639999   87.150002   88.150002  1536100.0
2022-05-06   87.489998   87.489998   84.419998   85.919998  3168000.0
2022-05-09   84.339996   84.680000   80.820000   81.209999  3072000.0
2022-05-10   82.839996   83.570000   79.599998   81.650002  4029100.0

[89 rows x 5 columns]

In [11]:
iws_prices = reader.get_data_yahoo(ticker3, start, end)
display(iws_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,IWS,IWS,IWS,IWS,IWS,IWS
Date,,,,,,
2022-01-03,121.871330,122.339996,122.919998,121.589996,122.500000,1215800.0
2022-01-04,123.116539,123.589996,123.959999,122.989998,122.989998,537700.0
2022-01-05,121.203896,121.669998,124.279999,121.629997,123.809998,581800.0
2022-01-06,121.741829,122.209999,122.910004,121.459999,122.129997,490400.0
2022-01-07,121.692024,122.160004,122.769997,121.860001,122.320000,455300.0


In [12]:
iws_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
iws_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2022-01-03,121.871330,122.339996,122.919998,121.589996,122.500000,1215800.0
2022-01-04,123.116539,123.589996,123.959999,122.989998,122.989998,537700.0
2022-01-05,121.203896,121.669998,124.279999,121.629997,123.809998,581800.0
2022-01-06,121.741829,122.209999,122.910004,121.459999,122.129997,490400.0
2022-01-07,121.692024,122.160004,122.769997,121.860001,122.320000,455300.0


In [13]:
iws_prices = iws_prices[["open", "high", "low", "close", "volume", "adj close"]]
iws_prices.drop(columns="adj close", axis=1, inplace=True)
iws_prices.head()

,open,high,low,close,volume
Date,,,,,
2022-01-03,122.500000,122.919998,121.589996,122.339996,1215800.0
2022-01-04,122.989998,123.959999,122.989998,123.589996,537700.0
2022-01-05,123.809998,124.279999,121.629997,121.669998,581800.0
2022-01-06,122.129997,122.910004,121.459999,122.209999,490400.0
2022-01-07,122.320000,122.769997,121.860001,122.160004,455300.0


In [14]:
iws_only_prices = pd.concat([iws_prices], axis=1, keys=['IWS'])
iws_only_prices

IWS                                               
                  open        high         low       close     volume
Date                                                                 
2022-01-03  122.500000  122.919998  121.589996  122.339996  1215800.0
2022-01-04  122.989998  123.959999  122.989998  123.589996   537700.0
2022-01-05  123.809998  124.279999  121.629997  121.669998   581800.0
2022-01-06  122.129997  122.910004  121.459999  122.209999   490400.0
2022-01-07  122.320000  122.769997  121.860001  122.160004   455300.0
...                ...         ...         ...         ...        ...
2022-05-04  114.190002  117.099998  113.209999  116.970001   981300.0
2022-05-05  116.000000  116.279999  112.519997  113.519997   527400.0
2022-05-06  113.040001  113.370003  111.099998  112.620003  1243800.0
2022-05-09  111.320000  111.519997  108.639999  109.000000   907000.0
2022-05-10  110.309998  110.610001  107.160004  108.709999   934800.0

[89 rows x 5 columns]

In [15]:
mgc_prices = reader.get_data_yahoo(ticker4, start, end)
display(mgc_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,MGC,MGC,MGC,MGC,MGC,MGC
Date,,,,,,
2022-01-03,168.836395,169.350006,169.380005,167.809998,168.669998,110700.0
2022-01-04,168.477493,168.990005,170.000000,168.300003,169.899994,79000.0
2022-01-05,165.157593,165.660004,169.020004,165.610001,168.699997,167600.0
2022-01-06,164.798676,165.300003,166.309998,164.389999,165.210007,140400.0
2022-01-07,164.240387,164.740005,165.669998,164.100006,165.369995,71100.0


In [16]:
mgc_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
mgc_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2022-01-03,168.836395,169.350006,169.380005,167.809998,168.669998,110700.0
2022-01-04,168.477493,168.990005,170.000000,168.300003,169.899994,79000.0
2022-01-05,165.157593,165.660004,169.020004,165.610001,168.699997,167600.0
2022-01-06,164.798676,165.300003,166.309998,164.389999,165.210007,140400.0
2022-01-07,164.240387,164.740005,165.669998,164.100006,165.369995,71100.0


In [17]:
mgc_prices = mgc_prices[["open", "high", "low", "close", "volume", "adj close"]]
mgc_prices.drop(columns="adj close", axis=1, inplace=True)
mgc_prices.head()

,open,high,low,close,volume
Date,,,,,
2022-01-03,168.669998,169.380005,167.809998,169.350006,110700.0
2022-01-04,169.899994,170.000000,168.300003,168.990005,79000.0
2022-01-05,168.699997,169.020004,165.610001,165.660004,167600.0
2022-01-06,165.210007,166.309998,164.389999,165.300003,140400.0
2022-01-07,165.369995,165.669998,164.100006,164.740005,71100.0


In [18]:
mgc_only_prices = pd.concat([mgc_prices], axis=1, keys=['MGC'])
mgc_only_prices

MGC                                              
                  open        high         low       close    volume
Date                                                                
2022-01-03  168.669998  169.380005  167.809998  169.350006  110700.0
2022-01-04  169.899994  170.000000  168.300003  168.990005   79000.0
2022-01-05  168.699997  169.020004  165.610001  165.660004  167600.0
2022-01-06  165.210007  166.309998  164.389999  165.300003  140400.0
2022-01-07  165.369995  165.669998  164.100006  164.740005   71100.0
...                ...         ...         ...         ...       ...
2022-05-04  145.919998  150.270004  144.600006  149.990005  346500.0
2022-05-05  148.360001  148.429993  143.000000  144.419998  243100.0
2022-05-06  143.600006  144.910004  141.610001  143.690002  237200.0
2022-05-09  141.369995  141.669998  138.369995  138.960007  285400.0
2022-05-10  141.240005  141.699997  137.990005  139.339996  313200.0

[89 rows x 5 columns]

In [19]:
iwc_prices = reader.get_data_yahoo(ticker5, start, end)
display(iwc_prices.head())

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,IWC,IWC,IWC,IWC,IWC,IWC
Date,,,,,,
2022-01-03,142.215683,142.339996,143.179993,140.559998,140.940002,115600.0
2022-01-04,141.706131,141.830002,143.600006,141.089996,143.059998,105200.0
2022-01-05,137.359924,137.479996,142.509995,137.169998,141.779999,76500.0
2022-01-06,137.589737,137.710007,138.429993,135.669998,137.669998,51600.0
2022-01-07,136.300858,136.419998,138.940002,136.300003,137.839996,45900.0


In [20]:
iwc_prices.columns = ["adj close", "close", "high", "low", "open", "volume"]
iwc_prices.head()

,adj close,close,high,low,open,volume
Date,,,,,,
2022-01-03,142.215683,142.339996,143.179993,140.559998,140.940002,115600.0
2022-01-04,141.706131,141.830002,143.600006,141.089996,143.059998,105200.0
2022-01-05,137.359924,137.479996,142.509995,137.169998,141.779999,76500.0
2022-01-06,137.589737,137.710007,138.429993,135.669998,137.669998,51600.0
2022-01-07,136.300858,136.419998,138.940002,136.300003,137.839996,45900.0


In [21]:
iwc_prices = iwc_prices[["open", "high", "low", "close", "volume", "adj close"]]
iwc_prices.drop(columns="adj close", axis=1, inplace=True)
iwc_prices.head()

,open,high,low,close,volume
Date,,,,,
2022-01-03,140.940002,143.179993,140.559998,142.339996,115600.0
2022-01-04,143.059998,143.600006,141.089996,141.830002,105200.0
2022-01-05,141.779999,142.509995,137.169998,137.479996,76500.0
2022-01-06,137.669998,138.429993,135.669998,137.710007,51600.0
2022-01-07,137.839996,138.940002,136.300003,136.419998,45900.0


In [22]:
iwc_only_prices = pd.concat([iwc_prices], axis=1, keys=['IWC'])
iwc_only_prices

IWC                                              
                  open        high         low       close    volume
Date                                                                
2022-01-03  140.940002  143.179993  140.559998  142.339996  115600.0
2022-01-04  143.059998  143.600006  141.089996  141.830002  105200.0
2022-01-05  141.779999  142.509995  137.169998  137.479996   76500.0
2022-01-06  137.669998  138.429993  135.669998  137.710007   51600.0
2022-01-07  137.839996  138.940002  136.300003  136.419998   45900.0
...                ...         ...         ...         ...       ...
2022-05-04  118.190002  120.459999  115.739998  120.010002   60700.0
2022-05-05  119.239998  119.239998  114.599998  115.690002   44000.0
2022-05-06  115.089996  115.260002  112.949997  113.800003   31000.0
2022-05-09  111.860001  112.070000  108.129997  108.699997   61600.0
2022-05-10  110.629997  111.050003  106.879997  108.820000   57400.0

[89 rows x 5 columns]

In [23]:
# Concatenate the ticker DataFrames
prices_btc_iwp = pd.concat([btc_prices, iwp_prices], axis=1, keys=["BTC-USD", "IWP"])
prices_btc_iws = pd.concat([btc_prices, iws_prices], axis=1, keys=["BTC-USD", "IWS"])
prices_btc_mgc = pd.concat([btc_prices, mgc_prices], axis=1, keys=["BTC-USD", "MGC"])
prices_btc_iwc = pd.concat([btc_prices, iwc_prices], axis=1, keys=["BTC-USD", "IWC"])

prices_btc_iwp = prices_btc_iwp.dropna()
prices_btc_iws = prices_btc_iws.dropna()
prices_btc_mgc = prices_btc_mgc.dropna()
prices_btc_iwc = prices_btc_iwc.dropna()

display(prices_btc_iwp.tail())
display(prices_btc_iws.tail())
display(prices_btc_mgc.tail())
prices_btc_iwc.tail()

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-05-04  37748.011719  39902.949219  37732.058594  39698.371094   
2022-05-05  39695.746094  39789.281250  35856.515625  36575.140625   
2022-05-06  36573.183594  36624.359375  35482.132812  36040.921875   
2022-05-09  34060.015625  34222.074219  30296.953125  30296.953125   
2022-05-10  30273.654297  32596.308594  29944.802734  31022.906250   

                               IWP                                              
                 volume       open       high        low      close     volume  
Date                                                                            
2022-05-04  36754404490  90.230003  92.779999  88.239998  92.660004  1767500.0  
2022-05-05  43106256317  91.639999  91.639999  87.150002  88.150002  1536100.0  
2022-05-06  37795577489  87.489998  87.489998  84.419998  85.919998  3168000.0  
2022-05-09  63355494961  84.339996  84.680000  80.820000  81.209999  3072000.0  
2022-05-10  59811038817  82.839996  83.570000  79.599998  81.650002  4029100.0

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-05-04  37748.011719  39902.949219  37732.058594  39698.371094   
2022-05-05  39695.746094  39789.281250  35856.515625  36575.140625   
2022-05-06  36573.183594  36624.359375  35482.132812  36040.921875   
2022-05-09  34060.015625  34222.074219  30296.953125  30296.953125   
2022-05-10  30273.654297  32596.308594  29944.802734  31022.906250   

                                IWS                                      \
                 volume        open        high         low       close   
Date                                                                      
2022-05-04  36754404490  114.190002  117.099998  113.209999  116.970001   
2022-05-05  43106256317  116.000000  116.279999  112.519997  113.519997   
2022-05-06  37795577489  113.040001  113.370003  111.099998  112.620003   
2022-05-09  63355494961  111.320000  111.519997  108.639999  109.000000   
2022-05-10  59811038817  110.309998  110.610001  107.160004  108.709999   

                       
               volume  
Date                   
2022-05-04   981300.0  
2022-05-05   527400.0  
2022-05-06  1243800.0  
2022-05-09   907000.0  
2022-05-10   934800.0

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-05-04  37748.011719  39902.949219  37732.058594  39698.371094   
2022-05-05  39695.746094  39789.281250  35856.515625  36575.140625   
2022-05-06  36573.183594  36624.359375  35482.132812  36040.921875   
2022-05-09  34060.015625  34222.074219  30296.953125  30296.953125   
2022-05-10  30273.654297  32596.308594  29944.802734  31022.906250   

                                MGC                                      \
                 volume        open        high         low       close   
Date                                                                      
2022-05-04  36754404490  145.919998  150.270004  144.600006  149.990005   
2022-05-05  43106256317  148.360001  148.429993  143.000000  144.419998   
2022-05-06  37795577489  143.600006  144.910004  141.610001  143.690002   
2022-05-09  63355494961  141.369995  141.669998  138.369995  138.960007   
2022-05-10  59811038817  141.240005  141.699997  137.990005  139.339996   

                      
              volume  
Date                  
2022-05-04  346500.0  
2022-05-05  243100.0  
2022-05-06  237200.0  
2022-05-09  285400.0  
2022-05-10  313200.0

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-05-04  37748.011719  39902.949219  37732.058594  39698.371094   
2022-05-05  39695.746094  39789.281250  35856.515625  36575.140625   
2022-05-06  36573.183594  36624.359375  35482.132812  36040.921875   
2022-05-09  34060.015625  34222.074219  30296.953125  30296.953125   
2022-05-10  30273.654297  32596.308594  29944.802734  31022.906250   

                                IWC                                      \
                 volume        open        high         low       close   
Date                                                                      
2022-05-04  36754404490  118.190002  120.459999  115.739998  120.010002   
2022-05-05  43106256317  119.239998  119.239998  114.599998  115.690002   
2022-05-06  37795577489  115.089996  115.260002  112.949997  113.800003   
2022-05-09  63355494961  111.860001  112.070000  108.129997  108.699997   
2022-05-10  59811038817  110.629997  111.050003  106.879997  108.820000   

                     
             volume  
Date                 
2022-05-04  60700.0  
2022-05-05  44000.0  
2022-05-06  31000.0  
2022-05-09  61600.0  
2022-05-10  57400.0

# PART II: MONTE CARLOS 

In [24]:
#MC_btc = MCSimulation(
 #   portfolio_data = btc_only_prices,
  #  weights = [1],
   # num_simulation = 500,
    #num_trading_days = 126)

# Review the simulation input data
#MC_btc.portfolio_data.head()

In [25]:
MC_iwp = MCSimulation(
    portfolio_data = iwp_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 45)

# Review the simulation input data
MC_iwp.portfolio_data.head()

IWP                                                 \
                  open        high         low       close     volume   
Date                                                                    
2022-01-03  115.550003  115.669998  113.230003  114.029999  1401700.0   
2022-01-04  114.160004  114.190002  111.059998  112.400002  1440300.0   
2022-01-05  112.019997  112.180000  108.019997  108.050003  2204800.0   
2022-01-06  107.779999  109.400002  106.559998  108.269997  2073000.0   
2022-01-07  108.169998  109.120003  106.459999  106.500000  2013200.0   

                         
           daily_return  
Date                     
2022-01-03          NaN  
2022-01-04    -0.014294  
2022-01-05    -0.038701  
2022-01-06     0.002036  
2022-01-07    -0.016348

In [26]:
MC_iws = MCSimulation(
    portfolio_data = iws_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 45)

# Review the simulation input data
MC_iws.portfolio_data.head()

IWS                                                 \
                  open        high         low       close     volume   
Date                                                                    
2022-01-03  122.500000  122.919998  121.589996  122.339996  1215800.0   
2022-01-04  122.989998  123.959999  122.989998  123.589996   537700.0   
2022-01-05  123.809998  124.279999  121.629997  121.669998   581800.0   
2022-01-06  122.129997  122.910004  121.459999  122.209999   490400.0   
2022-01-07  122.320000  122.769997  121.860001  122.160004   455300.0   

                         
           daily_return  
Date                     
2022-01-03          NaN  
2022-01-04     0.010217  
2022-01-05    -0.015535  
2022-01-06     0.004438  
2022-01-07    -0.000409

In [27]:
MC_mgc = MCSimulation(
    portfolio_data = mgc_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 45)

# Review the simulation input data
MC_mgc.portfolio_data.head()

MGC                                                \
                  open        high         low       close    volume   
Date                                                                   
2022-01-03  168.669998  169.380005  167.809998  169.350006  110700.0   
2022-01-04  169.899994  170.000000  168.300003  168.990005   79000.0   
2022-01-05  168.699997  169.020004  165.610001  165.660004  167600.0   
2022-01-06  165.210007  166.309998  164.389999  165.300003  140400.0   
2022-01-07  165.369995  165.669998  164.100006  164.740005   71100.0   

                         
           daily_return  
Date                     
2022-01-03          NaN  
2022-01-04    -0.002126  
2022-01-05    -0.019705  
2022-01-06    -0.002173  
2022-01-07    -0.003388

In [28]:
MC_iwc = MCSimulation(
    portfolio_data = iwc_only_prices,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 45)

# Review the simulation input data
MC_iwc.portfolio_data.head()

IWC                                                \
                  open        high         low       close    volume   
Date                                                                   
2022-01-03  140.940002  143.179993  140.559998  142.339996  115600.0   
2022-01-04  143.059998  143.600006  141.089996  141.830002  105200.0   
2022-01-05  141.779999  142.509995  137.169998  137.479996   76500.0   
2022-01-06  137.669998  138.429993  135.669998  137.710007   51600.0   
2022-01-07  137.839996  138.940002  136.300003  136.419998   45900.0   

                         
           daily_return  
Date                     
2022-01-03          NaN  
2022-01-04    -0.003583  
2022-01-05    -0.030671  
2022-01-06     0.001673  
2022-01-07    -0.009368

In [29]:
MC_btc_iwp = MCSimulation(
    portfolio_data = prices_btc_iwp,
    weights = [.1, .9],
    num_simulation = 1000,
    num_trading_days = 45)

# Review the simulation input data
MC_btc_iwp.portfolio_data.head()

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-01-03  47343.542969  47510.726562  45835.964844  46458.117188   
2022-01-04  46458.851562  47406.546875  45752.464844  45897.574219   
2022-01-05  45899.359375  46929.046875  42798.222656  43569.003906   
2022-01-06  43565.511719  43748.718750  42645.539062  43160.929688   
2022-01-07  43153.570312  43153.570312  41077.445312  41557.902344   

                                             IWP                          \
                 volume daily_return        open        high         low   
Date                                                                       
2022-01-03  33071628362          NaN  115.550003  115.669998  113.230003   
2022-01-04  42494677905    -0.012066  114.160004  114.190002  111.059998   
2022-01-05  36851084859    -0.050734  112.019997  112.180000  108.019997   
2022-01-06  30208048289    -0.009366  107.779999  109.400002  106.559998   
2022-01-07  84196607520    -0.037141  108.169998  109.120003  106.459999   

                                                
                 close     volume daily_return  
Date                                            
2022-01-03  114.029999  1401700.0          NaN  
2022-01-04  112.400002  1440300.0    -0.014294  
2022-01-05  108.050003  2204800.0    -0.038701  
2022-01-06  108.269997  2073000.0     0.002036  
2022-01-07  106.500000  2013200.0    -0.016348

In [30]:
MC_btc_iws = MCSimulation(
    portfolio_data = prices_btc_iws,
    weights = [.1, .9],
    num_simulation = 1000,
    num_trading_days = 45)

# Review the simulation input data
MC_btc_iws.portfolio_data.head()

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-01-03  47343.542969  47510.726562  45835.964844  46458.117188   
2022-01-04  46458.851562  47406.546875  45752.464844  45897.574219   
2022-01-05  45899.359375  46929.046875  42798.222656  43569.003906   
2022-01-06  43565.511719  43748.718750  42645.539062  43160.929688   
2022-01-07  43153.570312  43153.570312  41077.445312  41557.902344   

                                             IWS                          \
                 volume daily_return        open        high         low   
Date                                                                       
2022-01-03  33071628362          NaN  122.500000  122.919998  121.589996   
2022-01-04  42494677905    -0.012066  122.989998  123.959999  122.989998   
2022-01-05  36851084859    -0.050734  123.809998  124.279999  121.629997   
2022-01-06  30208048289    -0.009366  122.129997  122.910004  121.459999   
2022-01-07  84196607520    -0.037141  122.320000  122.769997  121.860001   

                                                
                 close     volume daily_return  
Date                                            
2022-01-03  122.339996  1215800.0          NaN  
2022-01-04  123.589996   537700.0     0.010217  
2022-01-05  121.669998   581800.0    -0.015535  
2022-01-06  122.209999   490400.0     0.004438  
2022-01-07  122.160004   455300.0    -0.000409

In [31]:
MC_btc_mgc = MCSimulation(
    portfolio_data = prices_btc_mgc,
    weights = [.1, .9],
    num_simulation = 1000,
    num_trading_days = 45)

# Review the simulation input data
MC_btc_mgc.portfolio_data.head()

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-01-03  47343.542969  47510.726562  45835.964844  46458.117188   
2022-01-04  46458.851562  47406.546875  45752.464844  45897.574219   
2022-01-05  45899.359375  46929.046875  42798.222656  43569.003906   
2022-01-06  43565.511719  43748.718750  42645.539062  43160.929688   
2022-01-07  43153.570312  43153.570312  41077.445312  41557.902344   

                                             MGC                          \
                 volume daily_return        open        high         low   
Date                                                                       
2022-01-03  33071628362          NaN  168.669998  169.380005  167.809998   
2022-01-04  42494677905    -0.012066  169.899994  170.000000  168.300003   
2022-01-05  36851084859    -0.050734  168.699997  169.020004  165.610001   
2022-01-06  30208048289    -0.009366  165.210007  166.309998  164.389999   
2022-01-07  84196607520    -0.037141  165.369995  165.669998  164.100006   

                                               
                 close    volume daily_return  
Date                                           
2022-01-03  169.350006  110700.0          NaN  
2022-01-04  168.990005   79000.0    -0.002126  
2022-01-05  165.660004  167600.0    -0.019705  
2022-01-06  165.300003  140400.0    -0.002173  
2022-01-07  164.740005   71100.0    -0.003388

In [32]:
MC_btc_iwc = MCSimulation(
    portfolio_data = prices_btc_iwc,
    weights = [.1, .9],
    num_simulation = 1000,
    num_trading_days = 45)

# Review the simulation input data
MC_btc_iwc.portfolio_data.head()

BTC-USD                                            \
                    open          high           low         close   
Date                                                                 
2022-01-03  47343.542969  47510.726562  45835.964844  46458.117188   
2022-01-04  46458.851562  47406.546875  45752.464844  45897.574219   
2022-01-05  45899.359375  46929.046875  42798.222656  43569.003906   
2022-01-06  43565.511719  43748.718750  42645.539062  43160.929688   
2022-01-07  43153.570312  43153.570312  41077.445312  41557.902344   

                                             IWC                          \
                 volume daily_return        open        high         low   
Date                                                                       
2022-01-03  33071628362          NaN  140.940002  143.179993  140.559998   
2022-01-04  42494677905    -0.012066  143.059998  143.600006  141.089996   
2022-01-05  36851084859    -0.050734  141.779999  142.509995  137.169998   
2022-01-06  30208048289    -0.009366  137.669998  138.429993  135.669998   
2022-01-07  84196607520    -0.037141  137.839996  138.940002  136.300003   

                                               
                 close    volume daily_return  
Date                                           
2022-01-03  142.339996  115600.0          NaN  
2022-01-04  141.830002  105200.0    -0.003583  
2022-01-05  137.479996   76500.0    -0.030671  
2022-01-06  137.710007   51600.0     0.001673  
2022-01-07  136.419998   45900.0    -0.009368

In [33]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
#MC_btc.calc_cumulative_return()

In [34]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_iwp.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.950887,1.007535,0.997147,1.006998,0.975099,1.026257,0.956977,0.985689,0.998200,1.037600,...,0.962155,1.008047,1.012092,0.978807,0.990665,1.001458,1.000155,0.983946,1.054561,0.988183
2,0.935676,1.014199,1.046389,0.978864,0.948880,0.994897,0.937796,0.959863,1.012310,1.030185,...,0.956608,1.009067,1.001233,0.976130,0.994848,0.988312,0.991833,0.988001,1.009705,0.985253
3,0.943548,1.009479,1.038742,0.967463,0.950526,0.997124,0.921827,0.960565,0.999482,1.043982,...,0.997476,0.980697,1.002358,0.967960,1.000245,1.001192,0.974615,0.990619,0.981964,0.994101
4,0.949109,1.002589,1.049172,0.962832,0.924822,0.996834,0.892316,0.946802,1.025538,0.997953,...,0.989167,0.972837,0.987140,0.970658,0.999449,0.984414,0.971636,0.987158,1.022520,0.988898
5,0.952736,0.992591,1.067371,0.989147,0.925476,0.982301,0.893916,0.943652,1.029689,1.008052,...,0.983488,0.943647,1.002713,0.952414,1.010954,0.979563,0.950409,0.958971,1.018392,0.920866
6,0.951356,0.979728,1.100686,0.951083,0.899623,1.027677,0.899968,0.974768,1.047528,1.003080,...,0.972818,0.963685,1.015335,0.965825,0.989288,0.940304,0.959349,0.981684,0.991012,0.926796
7,0.946390,0.969792,1.153840,0.954413,0.908663,1.088243,0.889968,0.953260,1.021206,0.994791,...,0.942399,0.981598,1.021441,0.904089,0.992507,0.900963,0.932164,0.966364,0.964438,0.917691
8,0.965552,0.975744,1.124516,0.953476,0.906396,1.111619,0.892186,0.961060,0.992420,0.960974,...,0.977962,0.998844,1.011701,0.910342,1.005539,0.904173,0.936595,0.976139,0.957496,0.941606
9,0.962311,0.963401,1.111918,0.933291,0.890249,1.112960,0.881384,0.969225,0.965014,0.946112,...,1.008930,0.978414,1.052529,0.886043,1.046513,0.916714,0.949039,0.967665,0.922877,0.946253


In [35]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_iws.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Monte Carlo simulation number 360.
Running Monte Carlo simulation number 370.
Running Monte Carlo simulation number 380.
Running Monte Carlo simulation number 390.
Running Monte Carlo simulation number 400.
Running Monte Carlo simulation number 410.
Running Monte Carlo simulation number 420.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.010493,1.003653,0.977067,0.993711,0.999102,0.988825,0.990763,1.017106,0.991214,1.003015,...,1.006545,0.979220,0.996711,0.980132,1.012515,0.984494,1.027232,1.013698,0.995732,1.005031
2,1.005013,0.981925,0.961721,1.000344,0.985379,0.970654,0.987631,1.027219,0.982542,1.015025,...,0.996101,0.976894,1.004472,0.979058,1.005310,1.001736,1.042515,1.017260,1.024476,1.014400
3,1.001152,0.995533,0.971560,0.982248,0.990833,0.995037,0.968496,1.019349,0.998611,1.021831,...,0.978287,0.960158,1.042405,0.973757,1.011784,0.995042,1.021190,1.012036,1.022351,1.009443
4,1.008270,0.980674,0.973498,0.965505,0.987261,0.988600,0.972149,0.999454,0.998676,1.024944,...,0.987557,0.974944,1.046959,0.986735,1.006960,0.977597,1.008733,0.996576,0.982106,0.981925
5,0.994719,0.974570,0.958640,0.978076,0.997882,0.988447,0.977091,1.005291,1.010873,1.039564,...,0.982833,0.973798,1.039300,0.978798,0.973517,0.985648,0.995115,0.999820,0.964171,0.990896
6,0.993378,0.974278,0.970530,0.973423,0.980561,0.956528,0.977630,1.005387,1.011317,1.024520,...,0.998460,1.016298,1.067452,0.967186,0.986991,0.983380,1.000337,0.992827,0.971681,0.994281
7,0.969882,0.982921,0.949125,0.971906,0.980773,0.965369,0.971847,1.022682,1.002138,1.010924,...,0.995316,0.993163,1.063877,0.993250,0.968656,0.996476,1.005696,1.010327,0.991048,0.982411
8,0.960159,0.969107,0.950054,0.973512,0.960181,0.975970,0.954975,1.011625,0.987290,0.998686,...,0.979208,0.986821,1.051462,0.986186,0.969593,0.991305,0.995510,1.019755,0.999520,0.978076
9,0.964347,0.972278,0.960785,0.968519,0.951592,0.973588,0.925417,1.027292,0.949241,1.033039,...,0.981317,0.990862,1.050060,0.988867,0.966658,1.020986,1.019573,1.010152,0.982923,0.961853


In [36]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_mgc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Monte Carlo simulation number 360.
Running Monte Carlo simulation number 370.
Running Monte Carlo simulation number 380.
Running Monte Carlo simulation number 390.
Running Monte Carlo simulation number 400.
Running Monte Carlo simulation number 410.
Running Monte Carlo simulation number 420.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.984725,0.984127,1.013516,0.989196,0.976376,0.977911,0.998609,1.010301,1.003012,1.024829,...,1.006520,0.998526,0.986643,0.996483,1.003265,0.971488,1.013108,0.971444,0.998997,0.985120
2,1.004159,0.984471,1.029946,0.986180,0.966470,0.967188,0.984278,1.026924,0.981257,1.027565,...,0.992242,1.010460,0.983962,0.973656,0.992994,1.002921,1.008504,0.984785,0.988705,0.978889
3,0.995622,0.991156,1.035835,0.976889,0.966401,0.956172,0.987350,1.037582,0.990463,1.008326,...,0.977293,0.998105,0.988061,0.966337,0.991483,0.951716,1.003931,0.980965,0.982335,0.987146
4,0.988302,1.008632,1.039096,0.973319,0.953310,0.934386,0.991290,1.055327,0.980387,1.001418,...,0.991457,1.011269,0.992953,0.979224,1.010439,0.962638,1.008554,0.978552,0.968448,0.972724
5,0.978806,1.000497,1.010239,0.930557,0.962799,0.923647,1.008437,1.048691,0.984177,1.009795,...,0.973550,0.981096,0.971441,0.994355,1.013631,0.979491,1.006591,0.969228,0.969536,0.964420
6,0.980461,1.000935,1.017106,0.925866,0.967478,0.936744,1.007887,1.052974,0.956428,1.007684,...,0.982806,0.975288,0.981718,0.985534,1.004017,0.993090,1.001953,0.949964,0.979071,0.962047
7,0.962290,1.001195,0.999208,0.946621,0.959876,0.939820,1.018978,1.071643,0.936051,1.013039,...,0.968122,0.982866,0.963358,0.986621,1.008272,0.999543,0.998155,0.939440,0.958551,0.997747
8,0.976647,0.987268,0.999999,0.959553,0.934910,0.910850,1.016194,1.065458,0.921749,1.019601,...,0.965205,0.953279,0.968557,0.993495,1.000631,0.971967,1.009119,0.931808,0.963708,0.984100
9,0.974905,1.000752,0.998542,0.975613,0.928127,0.921429,1.010063,1.029741,0.930240,0.993857,...,0.958996,0.959720,0.970691,0.995618,0.986504,0.989620,1.003201,0.920040,0.965890,0.978449


In [37]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_iwc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Monte Carlo simulation number 360.
Running Monte Carlo simulation number 370.
Running Monte Carlo simulation number 380.
Running Monte Carlo simulation number 390.
Running Monte Carlo simulation number 400.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.971063,0.994646,1.038206,1.037311,0.995139,0.991667,1.000139,0.976941,0.961491,1.000982,...,0.996984,1.000565,1.002388,0.972980,0.986755,1.015256,0.983503,0.995712,0.989548,0.979963
2,0.964322,0.989261,1.048492,1.045242,1.017656,1.011962,0.986785,0.958596,0.994951,1.000011,...,1.010194,0.988613,1.003751,0.968886,0.988507,1.014914,0.928504,0.993600,1.015740,1.004815
3,0.935445,0.976126,1.042036,1.046927,0.990070,1.033300,0.991913,0.962616,0.987476,0.966381,...,0.998886,0.982524,0.996775,0.969838,0.993160,1.008725,0.910911,0.997823,0.997589,1.005372
4,0.968893,0.955851,1.042273,1.001686,0.974789,1.032280,0.972279,0.965925,0.995427,0.947380,...,0.984234,0.999361,0.988997,0.956738,0.995240,1.035171,0.938255,1.020206,0.990852,0.991051
5,0.973514,0.962134,1.042999,0.979814,1.004083,1.005602,0.964421,1.008229,0.976589,0.949072,...,0.995923,0.988242,1.003378,0.933855,0.970854,1.047778,0.939351,1.035692,0.973929,0.995569
6,0.957625,0.956716,1.049855,0.960360,0.996534,0.994726,0.979371,1.007203,0.989310,0.949655,...,0.959328,0.977685,0.998665,0.915985,0.989232,1.027505,0.930721,1.074424,0.928590,0.992518
7,0.951209,0.945468,1.020207,0.961107,0.974373,0.988578,0.981458,0.976349,0.978591,0.924684,...,0.964813,0.969190,0.988844,0.900170,0.981567,1.040531,0.908583,1.086953,0.916285,1.002107
8,0.936259,0.962149,1.023800,0.966665,0.981432,0.975273,0.955303,0.991472,1.008854,0.930601,...,0.944671,0.947179,1.009533,0.910811,1.022094,1.010701,0.891003,1.096149,0.922534,0.971935
9,0.919738,0.986746,1.022788,0.957167,0.966327,0.957413,0.950384,0.976382,1.010800,0.929766,...,0.942499,0.928237,0.993584,0.885510,1.012453,1.017346,0.890351,1.095823,0.880478,0.970069


In [38]:
# Run the Monte Carlo simulation to forecast 6 mo. cumulative returns
MC_btc_iwp.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Monte Carlo simulation number 360.
Running Monte Carlo simulation number 370.
Running Monte Carlo simulation number 380.
Running Monte Carlo simulation number 390.
Running Monte Carlo simulation number 400.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.022871,0.977854,0.982072,0.993813,1.044779,0.992157,1.037674,0.999922,0.966470,1.012156,...,1.016764,1.013258,0.994731,1.021139,0.995062,1.029396,1.030619,0.983127,1.012661,1.016221
2,1.030357,0.992902,1.010498,0.978014,1.029431,1.001354,1.017237,0.983238,0.940901,1.031594,...,1.021808,0.998764,1.006678,0.983035,0.986440,1.051742,0.999118,0.981161,1.029174,1.028616
3,1.044302,1.020425,1.004896,1.002960,1.064486,1.013760,1.002122,0.958502,0.932607,1.011796,...,1.030622,0.976957,0.977610,0.945148,0.962744,1.040152,1.016430,1.030225,0.991960,1.050540
4,1.011043,1.017815,0.994605,1.022914,1.055094,0.984647,1.014960,0.939919,0.949517,1.011677,...,1.002903,0.975679,0.964374,0.975988,0.946076,1.047263,1.012722,0.996072,0.985592,1.053545
5,0.988586,1.013775,0.959297,1.011613,1.051418,0.987785,1.023731,0.956620,0.947298,1.004558,...,1.037872,0.995672,0.959648,0.958378,0.923072,1.059617,1.028580,0.990400,1.025544,1.078562
6,0.996127,1.009451,0.960280,0.995560,1.073930,0.993613,1.005419,0.942912,0.954317,1.025414,...,1.036545,1.020406,0.939749,0.948728,0.920127,1.023179,0.984388,0.995508,1.037642,1.067300
7,0.987965,1.008660,0.951796,0.986608,1.034993,1.036513,1.006842,0.924803,0.948176,1.034943,...,1.019374,1.013775,0.939649,0.940540,0.924373,1.009435,0.985315,1.028148,1.023455,1.073335
8,1.021131,1.016322,0.961385,0.971598,1.011380,1.085460,1.031873,0.942138,0.939179,1.023636,...,0.995755,0.982965,0.948113,0.945816,0.927393,1.006631,0.985061,1.005553,1.002778,1.029557
9,1.004023,1.034844,0.970493,0.940775,1.003384,1.071414,1.017378,0.907787,0.931935,0.973812,...,1.017073,1.002593,0.942450,0.969597,0.930470,1.010573,0.989607,1.051897,0.944640,1.028978


In [39]:
MC_btc_iws.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Monte Carlo simulation number 360.
Running Monte Carlo simulation number 370.
Running Monte Carlo simulation number 380.
Running Monte Carlo simulation number 390.
Running Monte Carlo simulation number 400.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.995319,1.008293,1.002306,1.000073,0.977132,1.019483,1.010179,0.973262,0.985196,1.000665,...,0.990169,0.982164,0.997814,0.982533,1.005047,0.997681,1.021799,1.004000,0.991701,0.987122
2,1.011126,1.021211,1.007104,0.998084,0.955928,1.012068,0.997691,0.986710,0.998495,1.013498,...,0.986531,0.973263,0.989179,0.989509,1.011685,0.992695,1.028855,1.012238,0.987193,0.997669
3,1.019215,1.009931,1.000877,0.987005,0.944767,1.023264,0.998676,0.981693,1.008100,1.013246,...,0.975721,0.955884,1.004220,0.959428,1.007363,0.996706,1.022993,1.001733,0.973843,1.002886
4,1.009519,1.011983,1.010471,0.976069,0.921227,1.001447,1.017145,0.978991,1.022776,1.015378,...,0.991816,0.942700,1.005996,0.972394,1.026433,1.000089,1.033989,0.974784,1.002330,1.013025
5,1.011637,1.019563,0.977890,0.980208,0.917341,0.995888,1.012769,0.984936,1.027133,1.007079,...,0.987946,0.964992,0.992985,0.954809,1.018170,0.990361,1.040765,0.981726,0.981691,1.029114
6,0.995944,1.028934,0.984387,0.959924,0.921568,0.996933,1.006276,1.011061,1.024857,1.000014,...,0.991887,0.961200,0.995033,0.965290,1.010543,0.996101,1.049972,0.978192,0.965707,1.009017
7,0.974804,1.043878,0.985114,0.949147,0.901551,1.006835,1.030309,1.041193,1.039466,1.006930,...,1.002493,0.982466,0.984945,0.961041,1.035728,0.982781,1.031984,0.970690,0.949120,0.997176
8,0.977891,1.039685,0.970073,0.930070,0.903835,1.020147,1.016037,1.034651,1.034654,1.005916,...,0.995694,0.975193,0.989623,0.962703,1.019945,0.996185,1.027437,0.951682,0.945709,0.995569
9,0.972215,1.035899,0.947863,0.939450,0.886661,1.028766,1.007705,1.014154,1.036432,1.024933,...,0.977754,0.969654,0.981920,0.982038,1.008427,1.007052,1.028899,0.947815,0.957872,0.989744


In [40]:
MC_btc_mgc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Monte Carlo simulation number 360.
Running Monte Carlo simulation number 370.
Running Monte Carlo simulation number 380.
Running Monte Carlo simulation number 390.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.999010,1.011260,0.996875,1.028502,0.978768,0.970727,0.973922,0.992732,0.984561,1.017005,...,0.981837,1.015312,1.002739,0.982023,0.978445,0.984255,0.972588,1.008522,0.994897,0.993731
2,0.992492,1.002082,0.992519,0.988648,0.964011,0.956011,0.963491,1.007218,0.979076,1.007550,...,0.992069,1.007463,1.023399,0.971896,0.971412,0.956138,0.968270,1.001259,0.982484,1.013457
3,0.972870,0.984003,0.991004,0.989185,0.994028,0.932182,0.956925,0.997651,0.959545,1.009912,...,0.966226,0.988668,1.023775,0.971545,0.978969,0.933301,0.965859,1.007313,0.971698,1.013413
4,0.974010,0.973908,0.989450,0.989130,1.000996,0.945313,0.957033,1.004308,0.952583,1.015342,...,0.958312,0.992681,1.015872,0.966582,0.982179,0.924288,0.974480,0.969319,0.956584,1.025850
5,0.985942,0.962052,0.965127,0.998831,0.999380,0.954131,0.932004,1.002155,0.938718,1.009281,...,0.951684,1.023629,1.018904,0.978088,0.955235,0.878150,0.974156,0.988484,0.914822,1.001611
6,0.985973,0.942268,0.969714,1.007503,0.974253,0.948510,0.913482,0.975045,0.931804,1.002894,...,0.967564,1.021891,1.031043,0.966498,0.981518,0.869798,0.983844,0.967492,0.906681,0.994072
7,0.992195,0.949614,0.933417,1.012504,0.969856,0.950691,0.915419,0.963269,0.915209,1.009411,...,0.985327,1.001284,1.010246,0.964165,0.962372,0.862573,0.974868,0.961345,0.924557,0.989176
8,0.990612,0.952703,0.939696,1.001439,0.966991,0.929909,0.905165,0.925341,0.927923,1.009737,...,0.987189,1.003743,0.995348,0.973860,0.967277,0.860290,0.968710,0.977290,0.922681,0.982217
9,1.007802,0.959652,0.913464,1.001071,0.982839,0.912566,0.892483,0.939205,0.939413,1.011187,...,0.999429,0.994415,0.971785,0.967156,0.973436,0.842856,0.971020,0.958615,0.922458,0.982240


In [41]:
MC_btc_iwc.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.


/Users/nicoleroberts/Dropbox/My Mac (Nicole’s MacBook Pro)/Desktop/AnalysisTeamCryvesto/Cryvesto/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Monte Carlo simulation number 350.
Running Monte Carlo simulation number 360.
Running Monte Carlo simulation number 370.
Running Monte Carlo simulation number 380.
Running Monte Carlo simulation number 390.
Running Monte Carlo simulation number 400.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.983872,0.977595,0.971516,0.998865,0.981306,1.006056,0.996230,1.014957,1.028476,0.991722,...,0.983596,0.973694,0.992601,1.015908,1.021737,1.022468,0.995772,1.021153,1.020918,1.024637
2,0.979070,0.962849,0.949837,0.991482,0.977767,1.000275,1.000886,0.987988,1.006273,0.988221,...,1.005724,0.961254,1.016712,0.983276,1.015410,1.022147,0.995190,1.009082,1.037838,1.010980
3,1.000750,0.957948,0.910726,0.984725,0.955789,1.003709,1.006594,0.982043,1.033891,0.980431,...,0.999394,0.957470,1.046861,0.975569,1.004056,1.000402,0.984753,0.984974,1.050320,1.014945
4,0.969297,0.965873,0.925892,0.979180,0.948748,0.998108,0.989473,0.968015,1.032283,0.976120,...,1.001200,0.942761,1.042683,0.985102,1.005037,0.992780,0.964607,0.969119,1.060857,1.030883
5,0.969438,0.946669,0.912689,0.996553,0.925333,0.981289,0.965652,0.977272,1.021118,0.988839,...,0.997810,0.930997,1.061059,0.973480,1.023185,0.988200,0.974451,0.974508,1.052843,1.046101
6,0.918359,0.916884,0.929706,0.989676,0.929771,1.002673,0.958906,0.930133,0.994396,1.001726,...,1.009901,0.953747,1.063148,0.984422,1.032410,0.977539,0.977486,0.985421,1.077648,1.047480
7,0.887111,0.926068,0.909378,1.011433,0.872236,1.009468,0.968396,0.942201,0.953480,0.983900,...,1.019167,0.953959,1.059173,0.968066,1.018098,1.001624,0.975056,0.977886,1.076006,1.048522
8,0.900345,0.949799,0.900761,1.015095,0.846710,1.002001,0.992158,0.951742,0.977249,0.988794,...,1.013696,0.933159,1.054200,0.957166,1.004009,0.972021,0.968346,0.974576,1.051750,1.059853
9,0.895582,0.931300,0.884563,1.011767,0.861741,1.003337,0.989275,0.947346,0.947378,0.964521,...,0.988681,0.925797,1.043353,0.937077,0.985321,0.949843,0.953910,0.985502,1.059446,1.024986


In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
#btc_line_plot = MC_btc.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
iwp_line_plot = MC_iwp.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
iws_line_plot = MC_iws.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
mgc_line_plot = MC_mgc.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
iwc_line_plot = MC_iwc.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
btc_iwp_line_plot = MC_btc_iwp.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
btc_iws_line_plot = MC_btc_iws.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
btc_mgc_line_plot = MC_btc_mgc.plot_simulation()

In [ ]:
# Visualize the 6 mo. Monte Carlo simulation by creating an overlay line plot
btc_iwc_line_plot = MC_btc_iwc.plot_simulation()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
#btc_distribution_plot = MC_btc.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
iwp_distribution_plot = MC_iwp.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
iws_distribution_plot = MC_iws.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
mgc_distribution_plot = MC_mgc.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
iwc_distribution_plot = MC_iwc.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
btc_iwp_distribution_plot = MC_btc_iwp.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
btc_iws_distribution_plot = MC_btc_iws.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
btc_mgc_distribution_plot = MC_btc_mgc.plot_distribution()

In [ ]:
# Visualize the probability distribution of the 6 mo. Monte Carlo simulation 
btc_iwc_distribution_plot = MC_btc_iwc.plot_distribution()

In [ ]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
#btc_table = MC_btc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
#print(btc_table)

In [42]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
iwp_table = MC_iwp.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(iwp_table)

count           1000.000000
mean               0.858271
std                0.124968
min                0.529017
25%                0.773921
50%                0.846214
75%                0.938313
max                1.337183
95% CI Lower       0.641761
95% CI Upper       1.135037
Name: 45, dtype: float64


In [43]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
iws_table = MC_iws.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(iws_table)

count           1000.000000
mean               0.945986
std                0.081239
min                0.708818
25%                0.890396
50%                0.940541
75%                1.001696
max                1.284189
95% CI Lower       0.796113
95% CI Upper       1.112642
Name: 45, dtype: float64


In [44]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
mgc_table = MC_mgc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(mgc_table)

count           1000.000000
mean               0.904375
std                0.096930
min                0.604739
25%                0.839846
50%                0.899403
75%                0.966847
max                1.346734
95% CI Lower       0.727898
95% CI Upper       1.105061
Name: 45, dtype: float64


In [45]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
iwc_table = MC_iwc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(iwc_table)

count           1000.000000
mean               0.873704
std                0.100629
min                0.585124
25%                0.803731
50%                0.865359
75%                0.939660
max                1.364420
95% CI Lower       0.697374
95% CI Upper       1.081423
Name: 45, dtype: float64


In [46]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_iwp_table = MC_btc_iwp.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_iwp_table)

count           1000.000000
mean               0.855455
std                0.117848
min                0.562801
25%                0.774042
50%                0.848119
75%                0.927202
max                1.413094
95% CI Lower       0.659389
95% CI Upper       1.130953
Name: 45, dtype: float64


In [47]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_iws_table = MC_btc_iws.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_iws_table)

count           1000.000000
mean               0.937729
std                0.078163
min                0.716859
25%                0.880463
50%                0.933961
75%                0.993255
max                1.165925
95% CI Lower       0.793684
95% CI Upper       1.099709
Name: 45, dtype: float64


In [48]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_mgc_table = MC_btc_mgc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_mgc_table)

count           1000.000000
mean               0.902080
std                0.087759
min                0.653657
25%                0.838065
50%                0.894661
75%                0.962111
max                1.194905
95% CI Lower       0.754879
95% CI Upper       1.089944
Name: 45, dtype: float64


In [49]:
# Generate summary statistics from the 6 mo. Monte Carlo simulation results
# Save the results as a variable
btc_iwc_table = MC_btc_iwc.summarize_cumulative_return()

# Review the 6 mo. Monte Carlo summary statistics
print(btc_iwc_table)

count           1000.000000
mean               0.879546
std                0.097889
min                0.639561
25%                0.808227
50%                0.874275
75%                0.943395
max                1.233183
95% CI Lower       0.705824
95% CI Upper       1.094762
Name: 45, dtype: float64


In [50]:
# Print the current balance of the stock and bond portion of the members portfolio
investment = 100000

In [51]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
#ci_lower_cumulative_return = round(btc_table[8] * investment, 2)
#ci_upper_cumulative_return = round(btc_table[9] * investment, 2)

# Print the result of your calculations
#print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
 #     f" with a 100% BTC over the next six months will end within the range of"
#      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

#mean = round(btc_table[1] * investment, 2)
#print(mean)

In [52]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(iwp_table[8] * investment, 2)
ci_upper_cumulative_return = round(iwp_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% IWP over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(iwp_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% IWP over the next six months will end within the range of $64176.06 and $113503.75.
85827.1


In [53]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(iws_table[8] * investment, 2)
ci_upper_cumulative_return = round(iws_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% IWS over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(iws_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% IWS over the next six months will end within the range of $79611.27 and $111264.23.
94598.56


In [54]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(mgc_table[8] * investment, 2)
ci_upper_cumulative_return = round(mgc_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% MGC over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(mgc_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% MGC over the next six months will end within the range of $72789.82 and $110506.07.
90437.45


In [55]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(iwc_table[8] * investment, 2)
ci_upper_cumulative_return = round(iwc_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 100% IWC over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(iwc_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 100% IWC over the next six months will end within the range of $69737.39 and $108142.27.
87370.43


In [56]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_iwp_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_iwp_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and IWP respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_iwp_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and IWP respectively over the next six months will end within the range of $65938.95 and $113095.26.
85545.52


In [57]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_iws_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_iws_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and IWS respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_iws_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and IWS respectively over the next six months will end within the range of $79368.42 and $109970.93.
93772.94


In [58]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_mgc_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_mgc_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and MGC respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_mgc_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and MGC respectively over the next six months will end within the range of $75487.9 and $108994.42.
90208.02


In [59]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes for the current stock/bond portfolio
ci_lower_cumulative_return = round(btc_iwc_table[8] * investment, 2)
ci_upper_cumulative_return = round(btc_iwc_table[9] * investment, 2)

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of ${investment} in a portfolio"
      f" with a 5/95 split between BTC and IWC respectively over the next six months will end within the range of"
      f" ${ci_lower_cumulative_return} and ${ci_upper_cumulative_return}.")

mean = round(btc_iwc_table[1] * investment, 2)
print(mean)

There is a 95% chance that an initial investment of $100000 in a portfolio with a 5/95 split between BTC and IWC respectively over the next six months will end within the range of $70582.37 and $109476.2.
87954.63
